In [1]:
import pandas as pd
from datetime import date
import requests,io
import json


In [2]:
url = "http://api.covid19india.org/csv/latest/cowin_vaccine_data_districtwise.csv"
metaurl = "https://raw.githubusercontent.com/mustafasaifee42/india-covid-vaccine-tracker/main/src/data/populationData.json"

In [3]:
data = requests.get(url)
meta = requests.get(metaurl)

In [4]:
populations = {}

In [5]:
distpops = meta.json()
distpops

[{'State_Name': 'Andaman and Nicobar Islands',
  'District_Name': 'Nicobars',
  'State_Letter_Code': 'AN',
  'District_Population': 41239},
 {'State_Name': 'Andaman and Nicobar Islands',
  'District_Name': 'South Andaman',
  'State_Letter_Code': 'AN',
  'District_Population': 222745},
 {'State_Name': 'Andaman and Nicobar Islands',
  'District_Name': 'North and Middle Andaman',
  'State_Letter_Code': 'AN',
  'District_Population': 166970},
 {'State_Name': 'Andhra Pradesh',
  'District_Name': 'Anantapur',
  'State_Letter_Code': 'AP',
  'District_Population': 4656080},
 {'State_Name': 'Andhra Pradesh',
  'District_Name': 'Chittoor',
  'State_Letter_Code': 'AP',
  'District_Population': 4758542},
 {'State_Name': 'Andhra Pradesh',
  'District_Name': 'Y.S.R. Kadapa',
  'State_Letter_Code': 'AP',
  'District_Population': 3288930},
 {'State_Name': 'Andhra Pradesh',
  'District_Name': 'East Godavari',
  'State_Letter_Code': 'AP',
  'District_Population': 6021109},
 {'State_Name': 'Andhra Prades

In [6]:
delhidists = map(lambda x: x['District_Population'],list(filter(lambda x:x['State_Letter_Code']=='DL', distpops)))
delhisum = sum(delhidists)

sansdelhi = {}
for dist in distpops:
    if dist['State_Letter_Code']!="DL":
        sansdelhi[dist['District_Name']] = dist['District_Population']
sansdelhi["Delhi"] = delhisum

In [7]:
len(sansdelhi)
populations=sansdelhi

In [8]:
lines = data.text.replace("YSR District, Kadapa","YSR District Kadapa").split("\n")

In [9]:
def calcFrom(st):
    st = st.split("/")
    d0 = date(2021, 1, 16)
    d1 = date(int(st[2]), int(st[1]), int(st[0]))
    delta = d1 - d0
    return (delta.days)

In [10]:
N = len(lines[100].split(","))

In [11]:
N

1506

In [12]:
def getHeader(line:str):
    hdr = []
    ld = line.split(",")
    for i in range(14,N,10):
        if len(ld[i])>0:
            hdr.append(ld[i])
    return ",".join(hdr)

In [13]:
myheader="dist,st,"+getHeader(lines[0])

In [19]:
def lineExtractor(line:str):
    ld = line.split(",")
    distname = ld[5]
    stname = ld[1]
    kernels = [distname, stname]
    # 14 is the index where total covaxin appears first, 15 for covishield, then cycles of 10
    for i in range(14,N,10):
        covaxin = ld[i]
        covishield = ld[i+1]
        first = ld[i-5]
        second = ld[i-4]
        if len(covaxin)>0 and len(covishield)>0:
            mymetric = int(first)
            kernels.append(str(mymetric))
        else:
            kernels.append("0")
    # Weird formatting issue - drop the last two zeroes
    kernels = kernels[:len(kernels)]
    return ",".join(kernels)

In [20]:
stZ = [myheader]
for dist in lines[2:]:
    stZ.append(lineExtractor(dist))
fin = "\n".join(stZ)

In [21]:
filify = io.StringIO(fin)
df = pd.read_csv(filify, sep=",")

In [22]:
fin

'dist,st,16/01/2021,17/01/2021,18/01/2021,19/01/2021,20/01/2021,21/01/2021,22/01/2021,23/01/2021,24/01/2021,25/01/2021,26/01/2021,27/01/2021,28/01/2021,29/01/2021,30/01/2021,31/01/2021,01/02/2021,02/02/2021,03/02/2021,04/02/2021,05/02/2021,06/02/2021,07/02/2021,08/02/2021,09/02/2021,10/02/2021,11/02/2021,12/02/2021,13/02/2021,14/02/2021,15/02/2021,16/02/2021,17/02/2021,18/02/2021,19/02/2021,20/02/2021,21/02/2021,22/02/2021,23/02/2021,24/02/2021,25/02/2021,26/02/2021,27/02/2021,28/02/2021,01/03/2021,02/03/2021,03/03/2021,04/03/2021,05/03/2021,06/03/2021,07/03/2021,08/03/2021,09/03/2021,10/03/2021,11/03/2021,12/03/2021,13/03/2021,14/03/2021,15/03/2021,16/03/2021,17/03/2021,18/03/2021,19/03/2021,20/03/2021,21/03/2021,22/03/2021,23/03/2021,24/03/2021,25/03/2021,26/03/2021,27/03/2021,28/03/2021,29/03/2021,30/03/2021,31/03/2021,01/04/2021,02/04/2021,03/04/2021,04/04/2021,05/04/2021,06/04/2021,07/04/2021,08/04/2021,09/04/2021,10/04/2021,11/04/2021,12/04/2021,13/04/2021,14/04/2021,15/04/2021,1

In [23]:
df

,dist,st,16/01/2021,17/01/2021,18/01/2021,19/01/2021,20/01/2021,21/01/2021,22/01/2021,23/01/2021,...,05/06/2021,06/06/2021,07/06/2021,08/06/2021,09/06/2021,10/06/2021,11/06/2021,12/06/2021,13/06/2021,14/06/2021
0,Nicobars,AN,0,0,0,1,1,2,3,206,...,8649,8649,8649,8648,8668,8709,8720,8719,8719,8782
1,North and Middle Andaman,AN,0,0,0,0,0,33,244,390,...,32431,32431,32431,32431,32450,32741,32965,33332,33333,33641
2,South Andaman,AN,23,23,42,88,123,204,305,324,...,70891,70899,71427,71838,72089,72648,73165,73657,73659,74209
3,Anantapur,AP,287,504,778,1324,1969,2496,3302,4089,...,629153,629180,629836,630169,630318,659440,660558,679580,679659,700984
4,Chittoor,AP,424,1012,925,1798,2829,3859,4766,5964,...,744513,744761,746778,747593,748364,774514,778781,796446,796508,816390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
750,Purulia,WB,31,173,320,340,340,560,697,750,...,283098,283144,286827,291802,297522,302822,308343,312604,313210,316881
751,Birbhum,WB,1,1,76,174,174,250,620,625,...,190074,190991,193730,195497,197712,198803,201520,202392,202982,206416
752,South 24 Parganas,WB,14,14,163,477,477,477,1343,1372,...,636925,642730,655121,663908,668691,673666,687273,692550,696373,702782
753,Uttar Dinajpur,WB,0,0,101,287,287,477,1143,1174,...,246791,247002,255116,261380,267107,273014,279243,282542,282541,289188


In [24]:
delhirows = df[df.st=="DL"]

In [25]:
delhirows = delhirows.groupby(["st"],as_index=False).sum()

In [26]:
delhirows

,st,16/01/2021,17/01/2021,18/01/2021,19/01/2021,20/01/2021,21/01/2021,22/01/2021,23/01/2021,24/01/2021,...,05/06/2021,06/06/2021,07/06/2021,08/06/2021,09/06/2021,10/06/2021,11/06/2021,12/06/2021,13/06/2021,14/06/2021
0,DL,892,1005,1366,3041,3041,5308,5331,12312,12614,...,4371075,4385618,4419597,4445352,4471099,4521905,4583464,4638935,4650867,4690571


In [27]:
delhirows['dist']='Delhi'

In [28]:
delhirows

,st,16/01/2021,17/01/2021,18/01/2021,19/01/2021,20/01/2021,21/01/2021,22/01/2021,23/01/2021,24/01/2021,...,06/06/2021,07/06/2021,08/06/2021,09/06/2021,10/06/2021,11/06/2021,12/06/2021,13/06/2021,14/06/2021,dist
0,DL,892,1005,1366,3041,3041,5308,5331,12312,12614,...,4385618,4419597,4445352,4471099,4521905,4583464,4638935,4650867,4690571,Delhi


In [29]:
df = df[df.st!="DL"]

In [30]:
finaldf = df.append(delhirows,ignore_index=True)

In [31]:
finalest = finaldf.groupby(["dist"]).sum() #Apparently the whore website cowin has multiple entries for dist huh

In [32]:
finalest.columns = list(map(calcFrom, list(finalest.columns)))

In [33]:
jso = finalest.to_json()

In [34]:
with open("bundle.json","w") as f:
    finjo = json.loads(jso)
    json.dump(finjo,f)
    f.close()

In [35]:
nd = list(finjo['0'].keys())

In [36]:
od = list(populations.keys())

In [37]:
finalest

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
dist,,,,,,,,,,,,,,,,,,,,,
Adilabad,8,8,127,762,762,1057,1278,1278,1291,1502,...,53140,53945,54839,55667,56511,57266,58075,59001,59800,61165
Agar Malwa,199,199,360,360,491,646,646,646,646,758,...,64540,65341,68132,68129,70785,70899,70896,70882,70882,73592
Agra,28,28,13,13,13,13,424,494,696,947,...,455428,455584,467370,478562,488832,498564,509609,520928,521001,531126
Ahmedabad,42,207,212,499,501,1229,2208,5842,5983,9184,...,2086987,2128105,2167835,2208711,2240689,2274085,2309776,2357685,2392096,2435346
Ahmednagar,302,415,327,695,1244,1244,2060,2921,2944,4222,...,568767,569411,571136,573538,584720,589163,591999,601536,602360,609898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yadgir,93,105,212,213,364,364,738,766,791,1473,...,167712,169592,171912,173638,175272,179747,186258,191700,193813,197158
Yamunanagar,146,158,203,624,625,1286,1644,2195,2275,3295,...,223218,223304,225392,227018,229469,231976,234855,236852,237212,240114
Yanam,1,1,3,7,7,7,12,25,25,36,...,7960,8053,8166,8251,8439,8620,8787,9076,9348,9716


In [38]:
for index, row in finalest.transpose().iterrows():
    print(row)

dist
Adilabad         8
Agar Malwa     199
Agra            28
Ahmedabad       42
Ahmednagar     302
              ... 
Yadgir          93
Yamunanagar    146
Yanam            1
Yavatmal        86
Zunheboto       51
Name: 0, Length: 714, dtype: int64
dist
Adilabad         8
Agar Malwa     199
Agra            28
Ahmedabad      207
Ahmednagar     415
              ... 
Yadgir         105
Yamunanagar    158
Yanam            1
Yavatmal        86
Zunheboto       51
Name: 1, Length: 714, dtype: int64
dist
Adilabad       127
Agar Malwa     360
Agra            13
Ahmedabad      212
Ahmednagar     327
              ... 
Yadgir         212
Yamunanagar    203
Yanam            3
Yavatmal        55
Zunheboto       94
Name: 2, Length: 714, dtype: int64
dist
Adilabad       762
Agar Malwa     360
Agra            13
Ahmedabad      499
Ahmednagar     695
              ... 
Yadgir         213
Yamunanagar    624
Yanam            7
Yavatmal       198
Zunheboto      136
Name: 3, Length: 714, dtype: int64
dist

In [39]:
df1 = finalest.transpose()

In [40]:
pop = pd.DataFrame(list(populations.items()))

In [41]:
df2 = pop.set_index(0).transpose()

In [42]:
per = df1.div(df2.iloc[0]/100, axis='columns')

In [43]:
myres = per.transpose().to_json()

In [44]:
with open("bundle-per.json","w") as f:
    finjo = json.loads(myres)
    json.dump(finjo,f)
    f.close()